# End-to-End Data Cleaning Pipeline with Raha and Baran (Minimal and Sequential)
We build an end-to-end data cleaning pipeline with our configuration-free error detection and correction systems, Raha and Baran.

In [1]:
import pandas
# import IPython.display

import raha

## Error Detection with Raha

### 1. Instantiating the Detection Class
We first instantiate the `Detection` class.

In [2]:
app_1 = raha.Detection()

# How many tuples would you label?
app_1.LABELING_BUDGET = 20

# Would you like to see the logs?
app_1.VERBOSE = True

### 2. Instantiating the Dataset
We next load and instantiate the dataset object.

In [3]:
dataset_dictionary = {
    "name": "flights",
    "path": "./data_with_rules/flights/noise/flights-inner_error-10.csv",
    "clean_path": "./data_with_rules/flights/clean.csv"
}
d = app_1.initialize_dataset(dataset_dictionary)
d.dataframe.head()

,src,flight,sched_dep_time,act_dep_time,sched_arr_time,act_arr_time
0,aa,AA-3859-IAH-ORD,7:10 a.m.,7:16 a.m.,9:40 a.m.,9:32 a.m.
1,aa,AA-1733-ORD-PHX,7:45 p.m.,7:58 p.m.,10:30 p.m.,1:30 p.m.
2,panynj,AA-1640-MIA-MCO,6:30 p.m.,6:30 p.m.,7:55 p.m.,7:25 p.m.
3,aa,AA-518-MIA-JFK,6:40 a.m.,6:54 a.m.,9:25 a.m.,9:28 a.m.
4,aa,AA-3756-ORD-SLC,12:15 p.m.,12:41 p.m.,2:45 p.m.,2:50 p.m.


### 3. Running Error Detection Strategies
Raha runs (all or the promising) error detection strategies on the dataset. This step could take a while because all the strategies should be run on the dataset. 

In [4]:
app_1.run_strategies(d)

193 strategy profiles are collected.


I just load strategies' results as they have already been run on the dataset!


### 4. Generating Features
Raha then generates a feature vector for each data cell based on the output of error detection strategies. 

In [5]:
app_1.generate_features(d)

60 Features are generated for column 0.
63 Features are generated for column 1.
40 Features are generated for column 2.
40 Features are generated for column 3.
40 Features are generated for column 4.
40 Features are generated for column 5.


### 5. Building Clusters
Raha next builds a hierarchical clustering model for our clustering-based sampling approach.

In [6]:
app_1.build_clusters(d)

A hierarchical clustering model is built for column 0.
A hierarchical clustering model is built for column 1.
A hierarchical clustering model is built for column 2.
A hierarchical clustering model is built for column 3.
A hierarchical clustering model is built for column 4.
A hierarchical clustering model is built for column 5.


### 6. Interactive Tuple Sampling and Labeling
Raha then iteratively samples a tuple. We should label data cells of each sampled tuple.

In [7]:
while len(d.labeled_tuples) < app_1.LABELING_BUDGET:
    app_1.sample_tuple(d)
    if d.has_ground_truth:
        app_1.label_with_ground_truth(d)
    else:
        print("Label the dirty cells in the following sampled tuple.")
        sampled_tuple = pandas.DataFrame(data=[d.dataframe.iloc[d.sampled_tuple, :]], columns=d.dataframe.columns)
        # IPython.display.display(sampled_tuple)
        for j in range(d.dataframe.shape[1]):
            cell = (d.sampled_tuple, j)
            value = d.dataframe.iloc[cell]
            correction = input("What is the correction for value '{}'? Type in the same value if it is not erronous.\n".format(value))
            user_label = 1 if value != correction else 0
            d.labeled_cells[cell] = [user_label, correction]
        d.labeled_tuples[d.sampled_tuple] = 1

Tuple 314 is sampled.
Tuple 314 is labeled.
Tuple 1477 is sampled.
Tuple 1477 is labeled.
Tuple 2313 is sampled.
Tuple 2313 is labeled.
Tuple 168 is sampled.
Tuple 168 is labeled.
Tuple 1079 is sampled.
Tuple 1079 is labeled.
Tuple 702 is sampled.
Tuple 702 is labeled.
Tuple 1213 is sampled.
Tuple 1213 is labeled.
Tuple 905 is sampled.
Tuple 905 is labeled.
Tuple 1897 is sampled.
Tuple 1897 is labeled.
Tuple 1325 is sampled.
Tuple 1325 is labeled.
Tuple 1850 is sampled.
Tuple 1850 is labeled.
Tuple 357 is sampled.
Tuple 357 is labeled.
Tuple 2149 is sampled.
Tuple 2149 is labeled.
Tuple 2262 is sampled.
Tuple 2262 is labeled.
Tuple 1374 is sampled.
Tuple 1374 is labeled.
Tuple 1267 is sampled.
Tuple 1267 is labeled.
Tuple 1438 is sampled.
Tuple 1438 is labeled.
Tuple 1371 is sampled.
Tuple 1371 is labeled.
Tuple 1708 is sampled.
Tuple 1708 is labeled.
Tuple 550 is sampled.
Tuple 550 is labeled.


### 7. Propagating User Labels
Raha then propagates each user label through its cluster.

In [8]:
app_1.propagate_labels(d)

The number of labeled data cells increased from 120 to 9858.


In [9]:
a = list(set([t[0] for t in list(d.extended_labeled_cells.keys())]))
b = d.extended_labeled_cells.keys()

### 8. Predicting Labels of Data Cells
Raha then trains and applies one classifier per data column to predict the label of the rest of data cells.

In [10]:
app_1.predict_labels(d)

A classifier is trained and applied on column 0.
A classifier is trained and applied on column 1.
A classifier is trained and applied on column 2.
A classifier is trained and applied on column 3.
A classifier is trained and applied on column 4.
A classifier is trained and applied on column 5.


### 9. Storing Results
Raha can also store the error detection results.

### 10. Evaluating the Error Detection Task
We can finally evaluate our error detection task.

In [11]:
p, r, f = d.get_data_cleaning_evaluation(d.detected_cells)[:3]
print("Raha's performance on {}:\nPrecision = {:.2f}\nRecall = {:.2f}\nF1 = {:.2f}".format(d.name, p, r, f))

Raha's performance on flights:
Precision = 0.12
Recall = 0.10
F1 = 0.11


# Error Correction with Baran

### 1. Instantiating the Correction Class
We first instantiate the `Correction` class.

In [12]:
app_2 = raha.Correction()

# How many tuples would you label?
app_2.LABELING_BUDGET = 20

# Would you like to see the logs?
app_2.VERBOSE = True

### 2. Initializing the Dataset Object
We next initialize the dataset object.

In [13]:
d = app_2.initialize_dataset(d)
d.dataframe.head()

,src,flight,sched_dep_time,act_dep_time,sched_arr_time,act_arr_time
0,aa,AA-3859-IAH-ORD,7:10 a.m.,7:16 a.m.,9:40 a.m.,9:32 a.m.
1,aa,AA-1733-ORD-PHX,7:45 p.m.,7:58 p.m.,10:30 p.m.,1:30 p.m.
2,panynj,AA-1640-MIA-MCO,6:30 p.m.,6:30 p.m.,7:55 p.m.,7:25 p.m.
3,aa,AA-518-MIA-JFK,6:40 a.m.,6:54 a.m.,9:25 a.m.,9:28 a.m.
4,aa,AA-3756-ORD-SLC,12:15 p.m.,12:41 p.m.,2:45 p.m.,2:50 p.m.


### 3. Initializing the Error Corrector Models
Baran initializes the error corrector models.

In [14]:
app_2.initialize_models(d)

The error corrector models are initialized.


### 4. Interactive Tuple Sampling, Labeling, Model updating, Feature Generation, and Correction Prediction
Baran then iteratively samples a tuple. We should label data cells of each sampled tuple. It then udpates the models accordingly and generates a feature vector for each pair of a data error and a correction candidate. Finally, it trains and applies a classifier to each data column to predict the final correction of each data error. Since we already labeled tuples for Raha, we use the same labeled tuples and do not label new tuples here.

In [15]:
# while len(d.labeled_tuples) < app_2.LABELING_BUDGET:
#     app_2.sample_tuple(d)
#     if d.has_ground_truth:
#         app_2.label_with_ground_truth(d)
#     else:
#         print("Label the dirty cells in the following sampled tuple.")
#         sampled_tuple = pandas.DataFrame(data=[d.dataframe.iloc[d.sampled_tuple, :]], columns=d.dataframe.columns)
#         IPython.display.display(sampled_tuple)
#         for j in range(d.dataframe.shape[1]):
#             cell = (d.sampled_tuple, j)
#             value = d.dataframe.iloc[cell]
#             correction = input("What is the correction for value '{}'? Type in the same value if it is not erronous.\n".format(value))
#             user_label = 1 if value != correction else 0
#             d.labeled_cells[cell] = [user_label, correction]
#         d.labeled_tuples[d.sampled_tuple] = 1
#     app_2.update_models(d)
#     app_2.generate_features(d)
#     app_2.predict_corrections(d)

for si in d.labeled_tuples:
    d.sampled_tuple = si
    app_2.update_models(d)
    app_2.generate_features(d)
    app_2.predict_corrections(d)

The error corrector models are updated with new labeled tuple 314.
85151 pairs of (a data error, a potential correction) are featurized.
56% (626 / 1123) of data errors are corrected.
The error corrector models are updated with new labeled tuple 1477.
85172 pairs of (a data error, a potential correction) are featurized.
56% (630 / 1123) of data errors are corrected.
The error corrector models are updated with new labeled tuple 2313.
87462 pairs of (a data error, a potential correction) are featurized.
56% (630 / 1123) of data errors are corrected.
The error corrector models are updated with new labeled tuple 168.
87462 pairs of (a data error, a potential correction) are featurized.
56% (630 / 1123) of data errors are corrected.
The error corrector models are updated with new labeled tuple 1079.
87462 pairs of (a data error, a potential correction) are featurized.
56% (630 / 1123) of data errors are corrected.
The error corrector models are updated with new labeled tuple 702.
87462 pair

### 5. Storing Results
Baran can also store the error correction results.

In [16]:
app_2.store_results(d)

The results are stored in ./data_with_rules/flights/noise/raha-baran-results-flights/error-correction/correction.dataset.


### 6. Evaluating the Error Correction Task
We can finally evaluate our error correction task.

In [17]:
p, r, f = d.get_data_cleaning_evaluation(d.corrected_cells)[-3:]
print("Baran's performance on {}:\nPrecision = {:.2f}\nRecall = {:.2f}\nF1 = {:.2f}".format(d.name, p, r, f))

Baran's performance on flights:
Precision = 0.09
Recall = 0.07
F1 = 0.08
